In [2]:
gpu_id = '1'
t = 0.5
region_side_length = 4


import warnings
warnings.filterwarnings("ignore")
import os
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id

import torch
import time
from tqdm import tqdm

from torch.utils.data import DataLoader
from data_utils.Dataset import Dataset,TestDataset,ValDataset,Test_FPS_Dataset,TestDataset_real

from tra_val.evaluation import *
from tra_val.loss import *
from tra_val.tra_val import *
from model.FARN import FARN
data_dir= 'HRSC2016/'

## show_FPS

In [3]:
#load trained model
model_trained = FARN(resnet_type='resnet34', boxes_dx_dy = 4 * region_side_length).cuda()
model_trained.load_state_dict(torch.load('checkpoints_34/g1_FARN_region4_0.5_34_07map_0.8946_date_20210316_04_33_36'))
model_trained.eval()

test_fps_dataset = Test_FPS_Dataset(region_side_length,data_dir) #FPS test only with image
test_fps_Dataloader = DataLoader(test_fps_dataset, 8, shuffle=False,
                                               num_workers=8, pin_memory=True)

pixel_coordinates = torch.zeros(128,128,2).cuda()
print('pixel_coordinates constructing')
for i in tqdm(range(128)):
    for ii in range(128):
        pixel_coordinates[i,ii,:] = torch.tensor([ii,i])*4

 17%|█▋        | 22/128 [00:00<00:00, 213.24it/s]

pixel_coordinates constructing


100%|██████████| 128/128 [00:00<00:00, 229.53it/s]


In [4]:
FPS_test(model_trained,
         test_fps_Dataloader,
         pixel_coordinates,
         test_epoch=6,
         fliter_theshold=0.5,
         scores_theshold=0,
         nms_theshold=0.5,
         nms_saved_images_num = 20,
         get_score=False)

counting the number of images


  0%|          | 0/6 [00:00<?, ?it/s]

number of images: 453
testing FPS


100%|██████████| 6/6 [00:21<00:00,  3.63s/it]

Number of img processed： 2718
Second used： 21.772586
Fps: 124.83588306873607


## show_visual_pic

In [8]:
import numpy as np
import math
from PIL import Image, ImageDraw, ImageFont 
class draw_bbox:
    def __init__(self,img,bbox,label,width = 7,ffill='red'):
        self.img = img
        self.bbox = bbox
        self.label = label
        self.width = width
        self.ffill = ffill
    def __len__(self,):
        return len(self.bbox)
    def __rotate__(self,x,y,width,height,st):
        cosA = math.cos(st)
        sinA = math.sin(st)
        x1=x-0.5*width
        y1=y-0.5*height
        x0=x+0.5*width
        y0=y1
        x2=x1
        y2=y+0.5*height
        x3=x0
        y3=y2
        x0n= (x0 -x)*cosA -(y0 - y)*sinA + x
        y0n = (x0-x)*sinA + (y0 - y)*cosA + y
        x1n= (x1 -x)*cosA -(y1 - y)*sinA + x
        y1n = (x1-x)*sinA + (y1 - y)*cosA + y
        x2n= (x2 -x)*cosA -(y2 - y)*sinA + x
        y2n = (x2-x)*sinA + (y2 - y)*cosA + y
        x3n= (x3 -x)*cosA -(y3 - y)*sinA + x
        y3n = (x3-x)*sinA + (y3 - y)*cosA + y
        return x0n,y0n,x1n,y1n,x2n,y2n,x3n,y3n
    def get_bbox_img(self):
        self.img = Image.fromarray(self.img.astype('uint8')).convert('RGB')
        draw = ImageDraw.Draw(self.img)
        font_ = ImageFont.truetype('arialuni.ttf',20)
        for i in range(self.__len__()):
            x0n,y0n,x1n,y1n,x2n,y2n,x3n,y3n = self.__rotate__(self.bbox[i,0],self.bbox[i,1],
                                                         self.bbox[i,2],self.bbox[i,3],self.bbox[i,4])
            rand_color = int(np.random.rand(1)*255)
            draw.line([(x0n,y0n),(x1n,y1n),(x2n,y2n),(x3n,y3n),(x0n,y0n)],width=self.width,fill=self.ffill)
#             draw.text((x1n,y1n),str(self.label),font = font_,ffill=self.ffill)
        self.img = np.array(self.img)
        return self.img

In [9]:
def imshow_prediction(trained_model, dataset, data_index, pixel_coordinates, center_theshold = 0.7, scores_theshold = 0.7,num_theshold = 0.7, correct_num = 2, get_score = False, img_size = [2,2], width_drawbbox = 7,show_all = False, show_prediction = False, draw_gt = True):
    if correct_num <1:
        print('correct_num need to be more than 1')
        return
    
    data = dataset[data_index]
    ori_img = data[0].astype('int')
    img = data[1]
    bboxes = data[2]
    bboxes_count = data[3]
    id_ = data[4]
    bodies, centers, boxes, scores = data[5], data[6][0], data[7][0], data[8]
    H,W = data[9]
    o_H,o_W = 512,512
    
#     H = json_data['images'][data_index]['height']
#     W = json_data['images'][data_index]['width']
#     id_ = json_data['images'][data_index]['file_name'].split('.')[0]
    #show_image
#     img_plt = img.transpose(1,2,0)
#     img_plt = (((img_plt - np.min(img_plt))/(np.max(img_plt)-np.min(img_plt)))*255).astype('int')
    # plt.figure(figsize=(10,10))
    
    if show_all:
        plt.imshow(ori_img)
        plt.show()

    img_prediction = torch.FloatTensor(img).unsqueeze(0)
    mask,features= trained_model.cuda()(img_prediction.cuda())
    mask = mask.cpu().detach().numpy()
    if show_all:
        plt.imshow(mask[0,0,:,:])
        plt.show()
        plt.imshow(mask[0,1,:,:])
        plt.show()
    
    mask = torch.FloatTensor(mask).cuda()
    mm = (mask[0,1]>center_theshold).float()
    xx = (((features[0,5].detach().float())+pixel_coordinates[:,:,0])/4)
    yy = (((features[0,6].detach().float())+pixel_coordinates[:,:,1])/4)
    xx = (xx*(xx<128).float()*(xx>0).float()* mm).long()
    yy = (yy*(yy<128).float()*(yy>0).float()* mm).long()
    rr = torch.zeros(128,128)
    rr[yy,xx] +=1
    rr[0,0]=0
    if show_all:
        plt.imshow(rr)
        plt.show()
    if 1-get_score:
        for i in range(correct_num-1):
            mm = (rr==1).float().cuda()
            xx = (((features[0,5].detach().float())+pixel_coordinates[:,:,0])/4)
            yy = (((features[0,6].detach().float())+pixel_coordinates[:,:,1])/4)
            xx = (xx*(xx<128).float()*(xx>0).float()* mm).long()
            yy = (yy*(yy<128).float()*(yy>0).float()* mm).long()
            rr = torch.zeros(128,128)
            rr[yy,xx] = 1
            rr[0,0]=0
            if show_all:
                plt.imshow(rr)
                plt.show()
    else:
        for i in range(correct_num-1):
            if i ==0 :
                score_rr = mask[0,0]
            else:
                score_rr = rr
            ss = torch.sum((rr>0).float()).int()
            xx = (((features[0,5].round().detach().float())+pixel_coordinates[:,:,0])/4)
            yy = (((features[0,6].round().detach().float())+pixel_coordinates[:,:,1])/4)
            resl = torch.cat((xx.unsqueeze(2),yy.unsqueeze(2),score_rr.unsqueeze(2)),2)
            mass = (resl[:,:,:2]<128).float()*(resl[:,:,:2]>0).float()
            resl = (resl*(mass[:,:,0]*mass[:,:,1]*(rr>0).float().cuda()).unsqueeze(2)).view(-1,3)
            resl = resl[torch.argsort(resl[:,0],descending=True)]
            rr = np.zeros((128,128))
            for l in resl[:ss]:
                if rr[l[1].long(),l[0].long()] < l[2]:
                    rr[l[1].long(),l[0].long()] = l[2]
            rr = torch.tensor(rr).cuda().float()
            if show_all:
                plt.imshow(rr.cpu())
                plt.show()
    
    f = []
    f.append(pixel_coordinates[:,:,0]+features[0,0,:,:])
    f.append(pixel_coordinates[:,:,1]+features[0,1,:,:])
    f.append(features[0,2,:,:])
    f.append(features[0,3,:,:])
    f.append(features[0,4,:,:])
    f.append(rr.cuda())
    f = torch.stack(f).permute(1,2,0).view(-1,6)
    ll = f[torch.argsort(f[:,5])]
    if torch.sum(ll[:,5]>scores_theshold)>0:
        ll = ll[-torch.sum(ll[:,5]>scores_theshold):]
        boxes = ll[:,:6].detach().cpu().numpy()
        keep = rotate_gpu_nms(np.array(ll.detach().cpu()),num_theshold)
        result = keep[:20]
    else :
        result = []

    img_draw = ori_img
    print("The number of detected bounding boxes:",len(result))
    for i in range(len(result)):
        bb = np.zeros((5))
        prew = boxes[result[i]][2]
        preh = boxes[result[i]][3]
        prex = boxes[result[i]][0]
        prey = boxes[result[i]][1]
        prest = boxes[result[i]][4]
        prescore = boxes[result[i]][5]
        print('predicted bbox:','x:',round(prex, 2),'y:',round(prey, 2),'w:',round(prew, 2),'h:',round(preh, 2),'theta:',round(prest, 2), 'scores',round(prescore, 2))
        bb[:] = prex,prey,prew,preh,prest
        gg = bb[np.newaxis,:]
        
        gg = resize_bbox(gg, (o_H, o_W), (H, W))
        
        img_draw = draw_bbox(img_draw,gg,str(round(prescore, 2)),width_drawbbox,'yellow').get_bbox_img()

    boxv = bboxes[:bboxes_count][:,:5]
    if draw_gt:
        print("The number of real bounding boxes:",bboxes_count)
        for i in range(bboxes_count):
            boxvi = boxv[i]
            bb = np.zeros((5))
            prew = boxvi[2]
            preh = boxvi[3]
            prex = boxvi[0]
            prey = boxvi[1]
            prest = boxvi[4]
            print('Gt bbox:','x:',round(prex, 2),'y:',round(prey, 2),'w:',round(prew, 2),'h:',round(preh, 2),'theta:',round(prest,2),'scores',round(1.00, 2))
            bb[:] = prex,prey,prew,preh,prest
            gg = bb[np.newaxis,:]
            img_draw = draw_bbox(img_draw,gg,'',width_drawbbox,'green').get_bbox_img()
    if show_prediction:
        plt.figure(figsize=img_size)
        plt.imshow(img_draw)
        plt.show()

    plt.imsave('predicted_results/'+id_+'.jpg', img_draw.astype(np.uint8))


In [ ]:
testDataset_real = TestDataset_real(region_side_length,data_dir, split='test') #test dateset without resize_bbox
tes_realDataloader = DataLoader(testDataset_real, 1, shuffle=False,
                                               num_workers=8, pin_memory=True)

# imshow_label(test_dataset, 1, label_show = [1,1,1,1])
for i in tqdm(range(tes_realDataloader.__len__())):
    imshow_prediction(model_trained,
                      testDataset_real,
                      i,
                      pixel_coordinates,
                      center_theshold=0.5,
                      scores_theshold=0.1,
                      num_theshold = 0.5,
                      correct_num=2,
                      get_score=True,
                      img_size=[10,10],
                      width_drawbbox = 15,
                      show_all = False, 
                      show_prediction = False, 
                      draw_gt = False
                     )

## mAP test

In [11]:
def Validate_FARN(model, epoch, tes_Dataloader, pixel_coordinates, Log_path, bestmap_07, bestmap_12,IOU_theshold = 0.5, fliter_theshold = 0.5, scores_theshold = 0, nms_theshold = 0.7, nms_saved_images_num = 20, plot_PR = False, if_relative_c = True):
    '''♦♦♦♦♦♦♦♦♦♦Val♦♦♦♦♦♦♦♦♦♦'''
    pred_bboxes, pred_labels, pred_scores = list(), list(), list()
    gt_bboxes, gt_labels, gt_difficults = list(), list(), list()
    for i,(ori_img, img,bboxes,bboxes_count,_,bodies, centers, boxes, scores, (H,W)) in enumerate(tqdm(tes_Dataloader,desc='val')):
        img,bboxes,bboxes_count,bodies, centers, boxes, scores = img.cuda(),bboxes.cuda(),bboxes_count.cuda(),bodies.cuda(), centers.cuda(), boxes.cuda(), scores.cuda()
        H,W = float(H),float(W)
        o_H,o_W = 512,512
        
        mask,features = model(img)
        mask,features = mask.detach(), features.detach()
        
        if if_relative_c:
            pred_xy_fea_ = (features[0,5:7].round().float()+pixel_coordinates[:,:,0:2].permute(2,0,1))/4
            pred_xy_fea_ = (pred_xy_fea_>127).float()*127 + pred_xy_fea_*(pred_xy_fea_<=127).float()
            pred_xy_fea_ = pred_xy_fea_*(pred_xy_fea_>0).float()  
        else:
            pred_xy_fea_ = ((features[0,5:7].round().float())/4)
            
        mm = (mask[0,1]>fliter_theshold).float()
        pred_xy_fea = pred_xy_fea_ * mm
        pred_centers = torch.zeros(128,128).cuda()
        pred_centers[pred_xy_fea[1].long(),pred_xy_fea[0].long()] = 1
        pred_centers[0,0]=0

        sum_centers = torch.sum(pred_centers).int()
        resl = torch.cat((pred_xy_fea_[0].unsqueeze(2),pred_xy_fea_[1].unsqueeze(2),mask[0,0].unsqueeze(2)),2)
        resl = (resl*(pred_centers).unsqueeze(2)).view(-1,3)
        resl = resl[torch.argsort(resl[:,0],descending=True)]
        
        pred_centers = np.zeros((128,128))
        for xyscore_pixel in resl[:sum_centers]:
            if pred_centers[xyscore_pixel[1].long(),xyscore_pixel[0].long()] < xyscore_pixel[2]:
                pred_centers[xyscore_pixel[1].long(),xyscore_pixel[0].long()] = xyscore_pixel[2]
        pred_centers = torch.tensor(pred_centers).cuda().float()

        Pr_bboxes = []
        if if_relative_c:
            Pr_bboxes.append(pixel_coordinates[:,:,0]+features[0,0,:,:])
            Pr_bboxes.append(pixel_coordinates[:,:,1]+features[0,1,:,:])
        else:
            Pr_bboxes.append(features[0,0,:,:])
            Pr_bboxes.append(features[0,1,:,:])
        Pr_bboxes.append(features[0,2,:,:])
        Pr_bboxes.append(features[0,3,:,:])
        Pr_bboxes.append(features[0,4,:,:])
        Pr_bboxes.append(pred_centers)
        Pr_bboxes = torch.stack(Pr_bboxes).permute(1,2,0).view(-1,6)
        Pr_bboxes = Pr_bboxes[torch.argsort(Pr_bboxes[:,5])] #sorting
        Pr_bboxes = Pr_bboxes[-torch.sum(Pr_bboxes[:,5]>scores_theshold):] #selecting pixels
        Pr_bboxes = resize_bbox(np.array(Pr_bboxes.cpu()), (o_H, o_W), (H, W))
        Pr_bboxes = torch.FloatTensor(Pr_bboxes)
        Pr_bboxes_ = Pr_bboxes.clone()
        Pr_bboxes_[:, 5] = 1
        
        Gt_bboxes = np.array(bboxes[0,:bboxes_count,:5].cpu().unsqueeze(0))

        if torch.sum(Pr_bboxes[:,5]>scores_theshold)!=0:
            keep = rotate_gpu_nms(np.array(Pr_bboxes.cpu()),nms_theshold)
            predicted_all_bboxes = Pr_bboxes[:,:6].cpu()
            predicted_bboxes_id = keep[:nms_saved_images_num]
            predicted_bboxes = np.array(predicted_all_bboxes[predicted_bboxes_id].cpu().unsqueeze(0))

            pred_bboxes_ = list(predicted_bboxes[:,:,:5])
            pred_labels_ = list(np.zeros((1,len(predicted_bboxes[0]))))
            pred_scores_ = list(predicted_bboxes[:,:,5])
        else:
            preiou = np.zeros((0,0,0))
            pred_bboxes_ =list(np.zeros((1,0,5)))
            pred_labels_ = list(np.zeros((1,0)))
            pred_scores_ = list(np.zeros((1,0)))

        gt_bboxes += list(Gt_bboxes[:,:,:5])
        gt_labels += list(np.zeros((1,len(Gt_bboxes[0]))))
        gt_difficults += list(np.zeros((1,len(Gt_bboxes[0]))))
        pred_bboxes += pred_bboxes_
        pred_labels += pred_labels_
        pred_scores += pred_scores_

    #MAP VOC 07
    result = eval_detection_voc(
        pred_bboxes, pred_labels, pred_scores,
        gt_bboxes, gt_labels, gt_difficults,
        use_07_metric=True,iou_thresh = IOU_theshold)
    MAP_07 = result[0]['map']
    
    #MAP VOC 12
    result = eval_detection_voc(
        pred_bboxes, pred_labels, pred_scores,
        gt_bboxes, gt_labels, gt_difficults,
        use_07_metric=False,iou_thresh = IOU_theshold)
    MAP_12 = result[0]['map']

    print(('Epoch:'+str(epoch)+'   07_MAP:'+str(MAP_07)+'   12_MAP:'+str(MAP_12)+'   Best07_Map:'+str(bestmap_07)+'   Best12_Map:'+str(bestmap_12)+'\n'))
#     f = open(Log_path,'a')
#     f.write('Epoch:'+str(epoch)+'   07_MAP:'+str(MAP_07)+'   12_MAP:'+str(MAP_12)+'   Best07_Map:'+str(bestmap_07)+'   Best12_Map:'+str(bestmap_12)+'\n')
#     f.close()
    if plot_PR:
        PR = np.zeros((len(list(result[1][0])),2))
        PR[:,0] = np.array(list(result[1][0]))
        PR[:,1] = np.array(list(result[2][0]))
        PR = PR[PR[:,1].argsort()] 
        for i in tqdm(range(len(list(result[1][0])))):
            for ii in range(i):
                if PR[ii,0]<PR[i,0]:
                    PR[ii,0]=PR[i,0]
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set(xlim=[0, 1], ylim=[0, 1], title='PR',
               ylabel='Percision', xlabel='Recall')
        plot1 = plt.plot(PR[:,1], PR[:,0], 'r',label='original values')
        plt.show()
    return MAP_07, MAP_12

In [13]:
# load trained model
model_trained = FARN(resnet_type='resnet34', boxes_dx_dy = 4 * region_side_length).cuda()
model_trained.load_state_dict(torch.load('checkpoints_34/g1_FARN_region4_0.5_34_07map_0.8946_date_20210316_04_33_36'))
model_trained.eval()

testDataset_real = TestDataset_real(region_side_length,data_dir, split='test') #test dateset without resize_bbox
tes_realDataloader = DataLoader(testDataset_real, 1, shuffle=False,
                                               num_workers=8, pin_memory=True)

pixel_coordinates = torch.zeros(128,128,2).cuda()
print('pixel_coordinates constructing')
for i in tqdm(range(128)):
    for ii in range(128):
        pixel_coordinates[i,ii,:] = torch.tensor([ii,i])*4


Validate_FARN(model_trained,
               0,
               tes_realDataloader,
               pixel_coordinates,
               0,
               0,
               0,
               fliter_theshold=0.5,
               scores_theshold=0,
               nms_theshold=0.5,
               nms_saved_images_num=20,
               plot_PR=False,
              )

 17%|█▋        | 22/128 [00:00<00:00, 212.89it/s]

pixel_coordinates constructing


val: 100%|██████████| 453/453 [00:11<00:00, 39.46it/s]


Epoch:0   07_MAP:0.8946448698795642   12_MAP:0.8918846144473693   Best07_Map:0   Best12_Map:0



(0.8946448698795642, 0.8918846144473693)